In [1]:
""" Ingest rasterized AQ30 indicators to earthengine.
-------------------------------------------------------------------------------

Compare Aqueduct 30 vs 21 and create change tables.

Author: Rutger Hofste
Date: 20190522
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

TESTING = 0

SCRIPT_NAME = "Y2019M05D22_RH_AQ30VS21_Compare_Tables_V01"
OUTPUT_VERSION = 1


AQ21_GCS_INPUT_PATH = "gs://aqueduct30_v01/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/"
AQ30_GCS_INPUT_PATH = "gs://aqueduct30_v01/Y2019M05D21_RH_AQ30VS21_Rasterize_AQ30_EE_V01/output_V05/"

ec2_input_path_aq21 = "/volumes/data/{}/input_V{:02.0f}/aq21/".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_input_path_aq30 = "/volumes/data/{}/input_V{:02.0f}/aq30".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("AQ21_GCS_INPUT_PATH: " + AQ21_GCS_INPUT_PATH +
      "\nAQ30_GCS_INPUT_PATH: " + AQ30_GCS_INPUT_PATH )

AQ21_GCS_INPUT_PATH: gs://aqueduct30_v01/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/
AQ30_GCS_INPUT_PATH: gs://aqueduct30_v01/Y2019M05D21_RH_AQ30VS21_Rasterize_AQ30_EE_V01/output_V05/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2019M05D22 UTC 19:03


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_input_path_aq21}
!rm -r {ec2_input_path_aq30}
!rm -r {ec2_output_path}
!mkdir -p {ec2_input_path_aq21}
!mkdir -p {ec2_input_path_aq30}
!mkdir -p {ec2_output_path}

In [6]:
!gsutil -m cp -r {AQ21_GCS_INPUT_PATH} {ec2_input_path_aq21}

Copying gs://aqueduct30_v01/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/BWS_s.tif...
Copying gs://aqueduct30_v01/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/DEFAULT.tif...
Copying gs://aqueduct30_v01/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/SV_s.tif...
Copying gs://aqueduct30_v01/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/WSV_s.tif...
- [4/4 files][ 59.5 MiB/ 59.5 MiB] 100% Done                                    
Operation completed over 4 objects/59.5 MiB.                                     


In [7]:
!gsutil -m cp -r {AQ30_GCS_INPUT_PATH} {ec2_input_path_aq30}

Copying gs://aqueduct30_v01/Y2019M05D21_RH_AQ30VS21_Rasterize_AQ30_EE_V01/output_V05/bws_score.tif...
Copying gs://aqueduct30_v01/Y2019M05D21_RH_AQ30VS21_Rasterize_AQ30_EE_V01/output_V05/iav_score.tif...
Copying gs://aqueduct30_v01/Y2019M05D21_RH_AQ30VS21_Rasterize_AQ30_EE_V01/output_V05/owr_score.tif...
Copying gs://aqueduct30_v01/Y2019M05D21_RH_AQ30VS21_Rasterize_AQ30_EE_V01/output_V05/sev_score.tif...
Copying gs://aqueduct30_v01/Y2019M05D21_RH_AQ30VS21_Rasterize_AQ30_EE_V01/output_V05/owr_wf.tif...
\ [5/5 files][143.6 MiB/143.6 MiB] 100% Done                                    
Operation completed over 5 objects/143.6 MiB.                                    


In [10]:
import rasterio
import numpy as np

In [14]:
aq21_input_path = "{}/{}{}".format(ec2_input_path_aq21,"Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/","DEFAULT.tif")

In [13]:
aq21_input_path 

'/volumes/data/Y2019M05D22_RH_AQ30VS21_Compare_Tables_V01/input_V01/aq21//Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02//DEFAULT.tif'